In [10]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
import spotipy.util as util

In [2]:
# Reading our data stored in a csv
billboard = pd.read_csv('../original/Hot Stuff.csv')
audio_feats = pd.read_excel('../original/Hot 100 Audio Features.xlsx')

In [3]:
billboard.head(3)

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1963-0...,6/1/1963,11,Still,Bill Anderson,StillBill Anderson,1.0,17.0,11.0,8.0
1,http://www.billboard.com/charts/hot-100/1967-0...,1/7/1967,11,Coming Home Soldier,Bobby Vinton,Coming Home SoldierBobby Vinton,1.0,17.0,11.0,8.0
2,http://www.billboard.com/charts/hot-100/1971-0...,7/3/1971,11,She's Not Just Another Woman,The 8th Day,She's Not Just Another WomanThe 8th Day,1.0,17.0,11.0,8.0


In [4]:
audio_feats.head(3)

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.017,0.000016,0.179,0.623,80.002,4.0
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.371,0.000000,0.231,0.323,95.799,4.0


In [5]:
top = billboard[billboard['Week Position'] == 1]

# Merge top ten music with audio features to get a dataframe with the features of all the top 10 songs
audio_genre = top.merge(right=audio_feats, how='inner', on=['SongID'])[['WeekID', 'Week Position', 'Song_x', 'Performer_x', 'SongID', 'spotify_track_id']]

audio_genre['WeekID'] = (pd.to_datetime(audio_genre['WeekID']).dt.year)

In [6]:
song_to_keep = audio_genre[['WeekID','spotify_track_id','SongID']].groupby(['WeekID','spotify_track_id']).agg('count').sort_values(by='SongID').reset_index().drop_duplicates(['WeekID'],keep='last').spotify_track_id.values

In [7]:
df_song_playlist = audio_genre[audio_genre.spotify_track_id.isin(set(song_to_keep))].drop_duplicates('Song_x').sort_values(by='WeekID')

In [8]:
tracks_id = df_song_playlist.spotify_track_id.to_list()

In [11]:
client_id= "922e0977111543faae64a89c4cc49932"
client_secret= "6c7aea8528684421b1883ac051f10931"
username = 'khalfiyassine753'
scope = 'playlist-modify-public'

token = util.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri='http://localhost:8008/callback') 
sp = spotipy.Spotify(auth=token)

In [12]:
playlist_name = f"The most popular music since the 1950s"    
sp.user_playlist_create(username,name=playlist_name)

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0gRamPUqr3UNXX8Exqf4M6'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/0gRamPUqr3UNXX8Exqf4M6',
 'id': '0gRamPUqr3UNXX8Exqf4M6',
 'images': [],
 'name': 'The most popular music since the 1950s',
 'owner': {'display_name': 'khalfiyassine753',
  'external_urls': {'spotify': 'https://open.spotify.com/user/khalfiyassine753'},
  'href': 'https://api.spotify.com/v1/users/khalfiyassine753',
  'id': 'khalfiyassine753',
  'type': 'user',
  'uri': 'spotify:user:khalfiyassine753'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSxlZGM0ZDVhODAzNWIwZTVlYzgyYmE5NjQ1YWJmZTU5ZjQyNzJiYjUx',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/0gRamPUqr3UNXX8Exqf4M6/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:0gRamPUqr3UNXX8Exqf

In [13]:
def GetPlaylistID(username, playlist_name):
    playlist_id = ''
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:  # iterate through playlists I follow
        if playlist['name'] == playlist_name:  # filter for newly created playlist
            playlist_id = playlist['id']
    print("Got Playlist ID.")
    return playlist_id

In [14]:
sp.user_playlist_add_tracks(username,GetPlaylistID(username,playlist_name), tracks_id)

Got Playlist ID.


{'snapshot_id': 'MixlNmJhYmFkMzEzOTU4YWNhZmYzZGU4OGU2Yjk5YzViOTU0ZGExMDU0'}